In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

In [2]:
iris=datasets.load_iris()
x=iris.data
y=iris.target

In [3]:
#spliting data into training and testing

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1)

In [4]:
d1=np.c_[x_train,y_train]
df=pd.DataFrame(d1,columns=['sepal_length','sepal_width','petal_length','petal_width','answer'])

In [5]:
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'answer'], dtype='object')

In [21]:
t=np.c_[x_test,y_test]
df_test=pd.DataFrame(t,columns=['sepal_length','sepal_width','petal_length','petal_width','answer'])

In [6]:
#if leaf_node is reached

def check_purity(data):
    label=data[:,-1]
    classes=np.unique(label)
    if len(classes)==1:
        return True
    else:
        return False

In [7]:
#for finding majority at a given stage(if stopping atmaximum depth is done)

def classify_data(data):
    label=data[:,-1]
    classes,counts=np.unique(data,return_counts=True)
    i=counts.argmax()
    c=classes[i]
    return c

In [8]:

#trial for unique function
classes,counts=np.unique(df.values[:,-1],return_counts=True)
len(counts)
classes[1]

1.0

In [9]:
#for dividing data  

def split_data(data,split_column,split_value):
    values=data[:,split_column]
    data_below=data[values<=split_value]
    data_above=data[values>split_value]
    
    
    return data_below,data_above

In [11]:
#for calculating entropy at a given level

def cal_entropy(data):
    label=data[:,-1]
    classes,counts=np.unique(data,return_counts=True)
    total=sum(counts)
    p=counts/total
    entropy=sum(p*(-np.log2(p)))
    
    return entropy
    


In [25]:
#for calculating total entropy obtained after spliting data

def total_entropy(data_below,data_above):
    total=len(data_below)+len(data_above)
    p_below=len(data_below)/total
    p_above=len(data_above)/total
    t_entropy=p_below*cal_entropy(data_below)+p_above*cal_entropy(data_above)
    
    return t_entropy

In [13]:
#testng individual performance of functions
a,b=split_data(df.values,3,0.8)
total_entropy(a,b)


5.114165214165116

In [14]:
#for finding probable values on which split is possible

def splits(data):
    s={}
    n=data.shape[1]
    for index in range(n-1):
        s[index]=[]
        values=data[:,index]
        unique_values=np.unique(values)
        
        for i in range(len(unique_values)):
            if i!=0:
                prev=unique_values[i-1]
                current=unique_values[i]
                split=(prev+current)/2
                s[index].append(split)
            
    return s

In [15]:
#finding best split value out of all probable values of split

def best_split(data,split):
    e=100000
    for index in split:
        for values in split[index]:
            data_below,data_above=split_data(data,split_column=index,split_value=values)
            t_entropy=total_entropy(data_below,data_above)
            if t_entropy<=e:
                e=t_entropy
                split_column=index
                split_value=values
    
    
    return split_column,split_value

In [16]:
#testng individual performance of functions
s=splits(df.values)
best_split(df.values,s)

(3, 0.8)

In [17]:
#combinig and building final tree

def decision_tree(df,counter=0,min_samples=5,max_depth=5):
    #level_no+=1
    if counter==0:
        global column_names
        column_names=df.columns
        data=df.values
    else:
        data=df
    #data=np.c_[x_train,y_train]
    #base_case
    if (check_purity(data))or (len(data)<min_samples)or(counter==max_depth):
        
        #print("Leaf Node Reached")
        #print(" Current Entropy is:",entropy(data))
        c=classify_data(data)
        return c
    else:
        counter+=1
        #recursion
        potential_splits=splits(data)
        best_split_column,best_split_value=best_split(data,potential_splits)
        data_below,data_above=split_data(data,best_split_column,best_split_value)
        
        feature_name=column_names[best_split_column]
        question="{} <= {}".format(feature_name,best_split_value)
        print(question)
        sub_tree={question:[]}
        #print(question)
        yes_answer=decision_tree(data_below,counter,min_samples,max_depth)
        c,counts=np.unique(data_below[:,-1],return_counts=True)
        print("level",counter)
        for i in range(len(counts)):
            print("Count of ",c[i],counts[i])
            
        print("Level done ")
        
        print("Current Entropy is:",total_entropy(data_below,data_above))
        no_answer=decision_tree(data_above,counter,min_samples,max_depth)
        sub_tree[question].append(yes_answer)
        sub_tree[question].append(no_answer)
        
    return sub_tree
        

In [18]:
tree=decision_tree(df)  
#entropy might seem to be in random order but actually it is getting decreased if checked levelwise

petal_width <= 0.8
level 1
Count of  0.0 37
Level done 
Current Entropy is: 5.114165214165116
petal_width <= 1.55
sepal_length <= 5.95
level 3
Count of  1.0 16
Level done 
Current Entropy is: 4.488996248524277
sepal_width <= 2.8499999999999996
petal_width <= 1.45
level 5
Count of  1.0 4
Count of  2.0 1
Level done 
Current Entropy is: 3.724372820753982
level 4
Count of  1.0 7
Count of  2.0 3
Level done 
Current Entropy is: 4.11306755882944
level 2
Count of  1.0 31
Count of  2.0 3
Level done 
Current Entropy is: 4.975898484152247
petal_width <= 1.95
sepal_length <= 6.05
sepal_length <= 5.85
level 5
Count of  2.0 3
Level done 
Current Entropy is: 3.3920053417460503
level 4
Count of  1.0 3
Count of  2.0 5
Level done 
Current Entropy is: 4.09114941838866
level 3
Count of  1.0 3
Count of  2.0 16
Level done 
Current Entropy is: 4.5555525194416076


In [19]:
#for finding classes of unknown samples

def predict(tree,sample):
    
    question=list(tree.keys())[0]
    feature_value,operator,value=question.split()
    if sample[feature_value]<=float(value):
        answer=tree[question][0]
    else:
        answer=tree[question][1]
        
    if not isinstance(answer,dict):
        
        return answer
    else:
        remaining_tree=answer
        return predict(remaining_tree,sample)
     

In [22]:
#testng individual performance of functions
s1=df_test.iloc[4]
print(s1)
predict(tree,s1)

sepal_length    7.9
sepal_width     3.8
petal_length    6.4
petal_width     2.0
answer          2.0
Name: 4, dtype: float64


2.0

In [23]:
#for calculating accuracy
def score(df,tree):
    pred=[]
    for i in range(len(df)):
        y_pred=predict(tree,df.iloc[i])
        pred.append(y_pred)
    acc=df.iloc[:,-1]==pred
    score=acc.mean()
    return score

In [24]:
s=score(df_test,tree)
s

0.9473684210526315